<a href="https://colab.research.google.com/github/Emivk/RNA-Digits/blob/master/RNAdigit_tf_comet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.2
    Uninstalling websocket-client-1.6.2:
      Successfully uninstalled websocket-client-1.6.2
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [3]:
import comet_ml
comet_ml.init(project_name="rnadigits")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [5]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/emivk/rnadigits/22f3e36faf724bc583bb7cc6c3540714



In [7]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
parameters = {
    "batch_size": 10,
    "epochs": 30,
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters)

In [8]:
model = Sequential()
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
model.add(Flatten()) #Otra forma de aplanar las imagenes
model.add(Dense(50, activation='sigmoid'))
#model.add(Dense(400, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
#model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
#model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
#model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
#model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
#model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
#model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 50)                39250     
                                                                 
 dense_1 (Dense)             (None, 10)                510       
                                                                 
Total params: 39760 (155.31 KB)
Trainable params: 39760 (155.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [10]:
model.compile(loss=parameters['loss'],optimizer=SGD(learning_rate=0.1,ema_momentum=0.9),metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
6000/6000 [==============================] - ETA: 0s - loss: 0.3812 - accuracy: 0.8946
Epoch 1: val_loss improved from inf to 0.23023, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 18s 3ms/step - loss: 0.3812 - accuracy: 0.8946 - val_loss: 0.2302 - val_accuracy: 0.9336
Epoch 2/30
  46/6000 [..............................] - ETA: 13s - loss: 0.2286 - accuracy: 0.9413

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5999/6000 [============================>.] - ETA: 0s - loss: 0.2004 - accuracy: 0.9419
Epoch 2: val_loss improved from 0.23023 to 0.16870, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 15s 2ms/step - loss: 0.2004 - accuracy: 0.9419 - val_loss: 0.1687 - val_accuracy: 0.9504
Epoch 3/30
5973/6000 [============================>.] - ETA: 0s - loss: 0.1532 - accuracy: 0.9558
Epoch 3: val_loss improved from 0.16870 to 0.13676, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 15s 2ms/step - loss: 0.1529 - accuracy: 0.9559 - val_loss: 0.1368 - val_accuracy: 0.9603
Epoch 4/30
5992/6000 [============================>.] - ETA: 0s - loss: 0.1267 - accuracy: 0.9635
Epoch 4: val_loss improved from 0.13676 to 0.11994, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 18s 3ms/step - loss: 0.1267 - accuracy: 0.9635 - val_loss: 0.1199 - val_accuracy: 0.9642
Epoch 5/30
5999/6000 [==========================

In [11]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=c34965fb084f4487a47653efe1a71316&experimentKey=22f3e36faf724bc583bb7cc6c3540714',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=c34965fb084f4487a47653efe1a71316&experimentKey=22f3e36faf724bc583bb7cc6c3540714',
 'assetId': 'c34965fb084f4487a47653efe1a71316'}

In [12]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/emivk/rnadigits/22f3e36faf724bc583bb7cc6c3540714
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.8946166634559631, 0.9961833357810974)
COMET INFO:     batch_accuracy [18000]         : (0.0, 1.0)
COMET INFO:     batch_loss [18000]             : (0.0004058735794387758, 2.8758559226989746)
COMET INFO:     epoch_duration [30]            : (14.373988728000313, 18.130294439999943)
COMET INFO:     loss [30]                      : (0.020822620019316673, 0.38116317987442017)
COMET INFO:     val_accuracy [30]              : (0.9336000084877014, 0.9767000079154968)
COMET INFO:     val_loss [30]